検証用・雑記用のnotebook

# モジュールのインポート

In [3]:
import pandas as pd
import glob
import os
from tqdm.notebook import tqdm

from modules.constants import LocalDirs
from modules import preparing
from modules import preprocessing
from modules import training
from modules import simulation
from modules import policies
%load_ext autoreload

SyntaxError: invalid syntax (_get_rawdata.py, line 182)

In [3]:
pd.read_pickle('data/raw/return_tables/return_tables_2021.pickle')

,0,1,2,3
202106030610,単勝,8,490,2
202106030610,複勝,8br3br6,190br170br170,4br2br3
202106030610,枠連,3 - 7,1090,5
202106030610,馬連,3 - 8,1330,3
202106030610,ワイド,3 - 8br6 - 8br3 - 6,510br400br660,3br1br11
...,...,...,...,...
202106030609,馬連,3 - 4,540,2
202106030609,ワイド,3 - 4br1 - 4br1 - 3,200br170br230,2br1br3
202106030609,馬単,4 → 3,730,2
202106030609,三連複,1 - 3 - 4,470,1


In [6]:
filename = os.path.join(LocalDirs.MASTER_DIR, 'horse_results_updated_at.csv')
pd.DataFrame(columns=['horse_id', 'updated_at']).to_csv(filename, index=None)

In [7]:
pd.read_csv(filename)

,horse_id,updated_at


In [10]:
files = glob.glob(os.path.join(LocalDirs.HTML_HORSE_DIR, '*.bin'))[:5]

In [15]:
import datetime

print(datetime.datetime.fromtimestamp(os.path.getmtime(files[0])))

2022-06-15 20:02:46.134937


In [16]:
datetime.datetime.now()

datetime.datetime(2022, 6, 23, 22, 59, 21, 759177)

In [17]:
datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2022-06-23 23:00:44'

In [19]:
master = pd.read_csv(filename)
master

,horse_id,updated_at


In [28]:
sample = pd.DataFrame({'horse_id': ['7878', '8989'], 'updated_at': '2021'})
sample

,horse_id,updated_at
0,7878,2021
1,8989,2021


In [32]:
sample2 = pd.DataFrame({'horse_id': ['8989', '3721']})
sample2

,horse_id
0,8989
1,3721


In [35]:
merged = sample.merge(sample2, how='outer')
merged

,horse_id,updated_at
0,7878,2021
1,8989,2021
2,3721,NaN


In [39]:
merged.loc[merged['horse_id'].isin(['7878', '3721']), :]

,horse_id,updated_at
0,7878,2021
2,3721,NaN


In [1]:
os.path.dirname('data/raw/horse_results/horse_results_2021.pickle')

'data/raw/horse_results'